### Document Splitting  하기
<목차>
1. RecursiveCharacterTextSplitter 과 CharacterTextSplitter
2. RecursiveCharacterTextSplitter 상세
   - text_splitter.split_documents(pages)
   - 실 생활 예제(Text Splitter)
3. Notion_db 분할(text_splitter.split_documents(notion_db))
4. Token splitting
5. Context aware splitting
   - 실 생활 예제(Context aware splitting)

<요약>
- RecursiveCharacterTextSplitter는 재귀적으로, CharacterTextSplitter는 단순 문자열를 분할하는 것으로 보인다.
- 문서를 청킹하는 경우가 실생활에서 더 많기 때문에 RecursiveCharacterTextSplitter 함수 활용을 더 많이 하길 권장
  - 아래의 메소드가 존재하며 실행활에서 문서를 로드하는 경우가 많기 때문에 split_documents()를 더 많이 활용하도록 한다
  - chunk_size, separator, chunk_overlap 순으로 최적화하여 활용하는 것이 중요한 것으로 보인다.  
   
create_documents(): 텍스트 리스트로 부터 도큐먼트 생성  
split_documents(): 도큐먼트 분할  
split_text(): 문자열 분할  

<심화>
- https://python.langchain.com/docs/modules/data_connection/document_transformers/
- https://python.langchain.com/docs/integrations/document_transformers

In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

#### 1. RecursiveCharacterTextSplitter 과 CharacterTextSplitter

In [3]:
chunk_size =26
chunk_overlap = 4

In [4]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
# 메소드
# create_documents(): 텍스트 리스트로 부터 도큐먼트 생성
# split_documents(): 도큐먼트 분할
# split_text(): 문자열 분할

In [5]:
text1 = 'abcdefghijklmnopqrstuvwxyz'
r_splitter.split_text(text1)

['abcdefghijklmnopqrstuvwxyz']

In [6]:
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'
r_splitter.split_text(text2) # chunk_overlap 개의 text가 overlap 되었다.

['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

In [7]:
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"
r_splitter.split_text(text3) # 공백 포함해서 chunk_size 단위로 분할

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

In [8]:
c_splitter.split_text(text3)

['a b c d e f g h i j k l m n o p q r s t u v w x y z']

In [9]:
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = ' ' # 
)
c_splitter.split_text(text3)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

#### 2. RecursiveCharacterTextSplitter 상세
- RecursiveCharacterTextSplitter가 일반 문서 분할에 추천된다.

In [11]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""
print(len(some_text))

496


In [12]:
c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator = ' '
)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", ""]
)

In [14]:
print(len(c_splitter.split_text(some_text)))
c_splitter.split_text(some_text)

2


['When writing documents, writers will use document structure to group content. This can convey to the reader, which idea\'s are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also,',
 'have a space.and words are separated by space.']

In [16]:
print(len(r_splitter.split_text(some_text)))
r_splitter.split_text(some_text)

2


["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.",
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [18]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "\. ", " ", ""]
)
print(len(r_splitter.split_text(some_text)))
r_splitter.split_text(some_text) # 분할 시 마침표(.)가 어색하게 분할되었다.

5


["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related",
 '. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns',
 '. Carriage returns are the "backslash n" you see embedded in this string',
 '. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [19]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
print(len(r_splitter.split_text(some_text)))
r_splitter.split_text(some_text)

5


["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related.",
 'For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns.',
 'Carriage returns are the "backslash n" you see embedded in this string.',
 'Sentences have a period at the end, but also, have a space.and words are separated by space.']

#### 실 생활 예제(Text Splitter)

In [20]:
from langchain.document_loaders import PyPDFLoader
filename = './[국문 G90 2023] genesis-g90-manual-kor-230601.pdf'
loader = PyPDFLoader(filename)
pages = loader.load()
print('len(pages): ', len(pages)) # <class 'langchain.schema.document.Document'> 의 리스트
pages_d = pages[24]

len(pages):  575


In [49]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

In [26]:
docs = text_splitter.split_documents([pages_d])
print('len(docs): ', len(docs))
print('len(pages_d): ', len([pages_d]))

len(docs):  2
len(pages_d):  1


In [35]:
print(docs[0].page_content[-40:])
print()
print(docs[1].page_content)

록 하십시오.
5. '찰칵' 소리가 날 때까지 버클에 밀어 넣으
십시오.

2. 안전벨트의 어깨띠가 어깨와 가슴을 지나
도록 하십시오.
3. 안전벨트가 꼬이거나 짓눌리지 않게 하십
시오.
4. 안전벨트의 골반띠가 골반을 부드럽게 지
나도록 하십시오.
5. '찰칵' 소리가 날 때까지 버클에 밀어 넣으
십시오.
탑승자 전원 안전벨트를 착용하십시오.
•3점식 안전벨트를 착용할 수 없는 어린이
의 경우 뒷좌석에 어린이용 보조 좌석을
장착해 앉히십시오.
RS4_G90_KO.book  Page 4


In [51]:
print(pages_d.page_content) # 전체 다 출력

2-4안전 및 주의 사항
올바른 운전 자세
올바른 운전 자세가 되도록 운전석과 스티어
링 휠을 조절하십시오.
바람직한 운전 자세는 좌석에 깊숙이 앉아
브레이크 페달을 끝까지 밟았을 때 무릎이
약간 굽혀지고, 손목이 스티어링 휠의 가장
먼 곳에 닿아야 합니다. 또한, 헤드레스트의
높이가 조절되는 차량인 경우는 운전자의 귀
상단이 헤드레스트 중심에 올 수 있도록 헤
드레스트를 조절하십시오.
좌석, 스티어링 휠, 미러 조
정
•좌석, 스티어링 휠, 미러는 출발 전에 조
절하시고 주행 중에 절대로 조작하지 마
십시오.
•내·외측의 미러를 조정하여, 충분한 시
야를 확보하십시오.
•모든 게이지 및 경고등을 확인하십시오.
•주차 브레이크를 풀고 브레이크 경고등이
꺼지는지 확인하십시오.
•차 주위에 사람이나 물체 등이 없는지 확
인하십시오.
҃Ҋ
 
•운전할 때 하이힐 등 운전하기 불편한 신
발을 신지 마십시오. 가속 페달, 브레이크
페달 등의 조작능력이 저하되어 사고의 원
인이 됩니다.
•주차 브레이크를 풀 때에는 차량이 움직일
수 있으므로 반드시 브레이크 페달을 확실
히 밟으십시오.
 운전석 주변 점검
•운전석 주변은 항상 깨끗하게 유지하십시
오. 빈 깡통 등이 페달 밑으로 굴러 들어갈
경우 페달 조작이 불가능하게 되어 매우
위험합니다.
•바닥 매트는 페달의 움직임을 방해하지 않
는 것으로 너무 두껍지 않으면서 바닥에
고정되는 제품이어야 합니다.
•차 안에는 화물을 좌석 높이 이상으로 적
재하지 마십시오.
안전벨트 착용
ORS021004
모든 좌석의 탑승자들은 가까운 거리라도주행 전에 반드시 안전벨트를 착용하십시오.
1. 엉덩이를 좌석 가장 안쪽으로 넣고 등을
등받이에 기대어 앉으십시오. 등을 구부리
거나 좌석 끝에 걸터앉지 마십시오. 
2. 안전벨트의 어깨띠가 어깨와 가슴을 지나
도록 하십시오.
3. 안전벨트가 꼬이거나 짓눌리지 않게 하십
시오.
4. 안전벨트의 골반띠가 골반을 부드럽게 지
나도록 하십시오.
5. '찰칵' 소리가 날 때까지 버클에 밀어 넣으
십시오.


In [58]:
text_splitter_r = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    separators=["(?<=\. )", " ", ""]
)

docs = text_splitter_r.split_documents([pages_d])
print('len(docs): ', len(docs))
print('len(pages_d): ', len([pages_d]))

len(docs):  2
len(pages_d):  1


In [59]:
print(docs[0].page_content)

2-4안전 및 주의 사항
올바른 운전 자세
올바른 운전 자세가 되도록 운전석과 스티어
링 휠을 조절하십시오.
바람직한 운전 자세는 좌석에 깊숙이 앉아
브레이크 페달을 끝까지 밟았을 때 무릎이
약간 굽혀지고, 손목이 스티어링 휠의 가장
먼 곳에 닿아야 합니다. 또한, 헤드레스트의
높이가 조절되는 차량인 경우는 운전자의 귀
상단이 헤드레스트 중심에 올 수 있도록 헤
드레스트를 조절하십시오.
좌석, 스티어링 휠, 미러 조
정
•좌석, 스티어링 휠, 미러는 출발 전에 조
절하시고 주행 중에 절대로 조작하지 마
십시오.
•내·외측의 미러를 조정하여, 충분한 시
야를 확보하십시오.
•모든 게이지 및 경고등을 확인하십시오.
•주차 브레이크를 풀고 브레이크 경고등이
꺼지는지 확인하십시오.
•차 주위에 사람이나 물체 등이 없는지 확
인하십시오.
҃Ҋ
 
•운전할 때 하이힐 등 운전하기 불편한 신
발을 신지 마십시오. 가속 페달, 브레이크
페달 등의 조작능력이 저하되어 사고의 원
인이 됩니다.
•주차 브레이크를 풀 때에는 차량이 움직일
수 있으므로 반드시 브레이크 페달을 확실
히 밟으십시오.
 운전석 주변 점검
•운전석 주변은 항상 깨끗하게 유지하십시
오. 빈 깡통 등이 페달 밑으로 굴러 들어갈
경우 페달 조작이 불가능하게 되어 매우
위험합니다.
•바닥 매트는 페달의 움직임을 방해하지 않
는 것으로 너무 두껍지 않으면서 바닥에
고정되는 제품이어야 합니다.
•차 안에는 화물을 좌석 높이 이상으로 적
재하지 마십시오.
안전벨트 착용
ORS021004
모든 좌석의 탑승자들은 가까운 거리라도주행 전에 반드시 안전벨트를 착용하십시오.
1. 엉덩이를 좌석 가장 안쪽으로 넣고 등을
등받이에 기대어 앉으십시오. 등을 구부리
거나 좌석 끝에 걸터앉지 마십시오. 
2. 안전벨트의 어깨띠가 어깨와 가슴을 지나
도록 하십시오.
3. 안전벨트가 꼬이거나 짓눌리지 않게 하십
시오.
4. 안전벨트의 골반띠가 골반을 부드럽게 지
나도록 하십시오.
5.


In [57]:
print(docs[1].page_content)

또한, 헤드레스트의
높이가 조절되는 차량인 경우는 운전자의 귀
상단이 헤드레스트 중심에 올 수 있도록 헤
드레스트를 조절하십시오.
좌석, 스티어링 휠, 미러 조
정
•좌석, 스티어링 휠, 미러는 출발 전에 조
절하시고 주행 중에 절대로 조작하지 마
십시오.
•내·외측의


#### 3. Notion_db 분할(text_splitter.split_documents(notion_db))

In [37]:
from langchain.document_loaders import NotionDirectoryLoader
loader = NotionDirectoryLoader("docs/Notion_DB")
notion_db = loader.load()

docs = text_splitter.split_documents(notion_db)

print(len(notion_db))
print(len(docs))

1
31


#### 4. Token splitting
- LLM은 토큰에 지정된 Context Windows가 있는 경우가 많기 때문에 유용할 수 있다.

In [40]:
from langchain.text_splitter import TokenTextSplitter

In [41]:
text1 = "foo bar bazzyfoo"
text_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)
text_splitter.split_text(text1)

['foo', ' bar', ' b', 'az', 'zy', 'foo']

In [43]:
text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0)
docs = text_splitter.split_documents([pages_d])

In [47]:
docs[:10] # 토큰으로 분할하니 난리다..

[Document(page_content='2-4안전 �', metadata={'source': './[국문 G90 2023] genesis-g90-manual-kor-230601.pdf', 'page': 24}),
 Document(page_content='�� 주의 사�', metadata={'source': './[국문 G90 2023] genesis-g90-manual-kor-230601.pdf', 'page': 24}),
 Document(page_content='�\n올바�', metadata={'source': './[국문 G90 2023] genesis-g90-manual-kor-230601.pdf', 'page': 24}),
 Document(page_content='� 운전 자', metadata={'source': './[국문 G90 2023] genesis-g90-manual-kor-230601.pdf', 'page': 24}),
 Document(page_content='세\n올바', metadata={'source': './[국문 G90 2023] genesis-g90-manual-kor-230601.pdf', 'page': 24}),
 Document(page_content='른 운전 �', metadata={'source': './[국문 G90 2023] genesis-g90-manual-kor-230601.pdf', 'page': 24}),
 Document(page_content='��세가 �', metadata={'source': './[국문 G90 2023] genesis-g90-manual-kor-230601.pdf', 'page': 24}),
 Document(page_content='�도록 운', metadata={'source': './[국문 G90 2023] genesis-g90-manual-kor-230601.pdf', 'page': 24}),
 Document(page_content='전석과 �', metadat

In [45]:
pages[0].metadata

{'source': './[국문 G90 2023] genesis-g90-manual-kor-230601.pdf', 'page': 0}

#### 5. Context aware splitting
- 청킹은 공통 맥락을 가진 텍스트를 함께 유지하는 것을 목표로 한다.
- 텍스트 분할에서는 관련 텍스트를 함께 유지하기 위해 `문장`이나 `기타 구분 기호`를 사용하는 경우가 많지만, 
  많은 문서(예: Markdown)에는 분할에 명시적으로 사용할 수 있는 구조(헤더)가 있습니다.
--> 

In [67]:
from langchain.document_loaders import NotionDirectoryLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter

markdown_document = """
# Title\n\n \

## Chapter 1\n\n \
Hi this is Jim\n\n 
Hi this is Joe\n\n \

### Section \n\n \
Hi this is Lance \n\n 

## Chapter 2\n\n \
Hi this is Molly"""

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [68]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)
md_header_splits = markdown_splitter.split_text(markdown_document)

In [69]:
md_header_splits[0]

Document(page_content='Hi this is Jim  \nHi this is Joe', metadata={'Header 1': 'Title', 'Header 2': 'Chapter 1'})

In [70]:
md_header_splits[1]

Document(page_content='Hi this is Lance', metadata={'Header 1': 'Title', 'Header 2': 'Chapter 1', 'Header 3': 'Section'})

#### 실 생활 예시(Context aware splitting)

In [71]:
loader = NotionDirectoryLoader("docs/Notion_DB")
docs = loader.load()
txt = ' '.join([d.page_content for d in docs])

In [72]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

In [73]:
md_header_splits = markdown_splitter.split_text(txt)

In [74]:
md_header_splits[0]

Document(page_content='### Authors  \nAndrew Brock  \nSoham De  \nSamuel L. Smith  \nKaren Simonyan  \nDeepMind  \n### Prerequisites  \n1. Batch Normalization  \n[https://arxiv.org/pdf/1502.03167.pdf](https://arxiv.org/pdf/1502.03167.pdf)  \n2. Brock et al. 2021. Characterizing Signal Propagation to Close the Performance Gap in Unnormalized ResNets ([https://arxiv.org/pdf/2101.08692.pdf](https://arxiv.org/pdf/2101.08692.pdf))\n- 상세 내용\n1. **Contribution**\n- forward pass의 시그널 전파를 시각화하는 분석 도구 제안 (SPP, Signal Propagation Plot)\n- BatchNorm 레이어 없이 높은 성능을 발휘하는 ResNet 설계  \n2. **주제 논문과의 공통점 & 차이점**\n- 공통점\n- Batch Normalization의 문제를 지적 - 학습 데이터 간의 독립성 훼손, 계산량 및 메모리 과부하, 예상치 못한 버그 유발, batch size에 따라 정확도 달라짐, 학습과 추론 간의 갭\n- Batch Normalization의 장점 나열 - loss surface를 스무스하게 해줌, 높은 learning rate로 학습 가능, 미니배치마다 다른 statistics로 인한 정규화 효과, skip connection 사용시 forward pass의 신호 전파가 잘 이루어지도록 해줌\n- BatchNorm 사용하지 않음\n- 차이점:\n- BatchNorm의 역할을 대신하는 Scaled Weight Standardization 제안\n- 근데 SOTA보다 못함.. 그래서 추후